In [ ]:
!pip install tweet-preprocessor
from google.colab import drive
drive.mount('/content/drive')
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

import keras
import tweepy
import pandas as pd
import preprocessor as p
import re
import random
import numpy as np
import operator
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


consumer_key = "i89JroFRgQPxgWQRG0u72mmWL" 
consumer_secret = "pEJLfkncfCPiRM6xt70YP7Ov9wJtR1hH8pFf0z96MaD2Zw7qJW" 
access_token = "1319347150958985216-wGOvb5BpH0h0E8eLfHoT2vk6blTcPZ" 
access_token_secret = "BeCyKDTZtSdhvYub3cQRTEYBu6yBxfJllEQ0wmwPpQigY" 
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#rmodel
file_obj1 = drive.CreateFile({'id': '1NpXiBXsc2asV_CA7GbFAK-8pemhgc6Tc'})
file_obj1.GetContentFile('rmodel.h5')
rmodel = keras.models.load_model('rmodel.h5')

#cmodel
file_obj2 = drive.CreateFile({'id': '1dqeQy8Ae8qF2vjgMwXypd3k-DjV35Zlo'})
file_obj2.GetContentFile('cmodel.h5')
cmodel = keras.models.load_model('cmodel.h5')

In [ ]:
cmodel_labels=['assistance','stress','survivorsofabuse','almosthomeless','anxiety','relationships','food_pantry','ptsd','domesticviolence','homeless']
rmodel_labels=['Safe to ignore','Concerned','Strongly Concerened']

In [ ]:
def extract_coordinates(row):
    if row['Tweet Coordinates']:
        return row['Tweet Coordinates']['coordinates']
    else:
        return None
def extract_place(row):
    if row['Place Info']:
        return row['Place Info'].full_name
    else:
        return None

def tokenization(text):
    text = re.split('\W+', text)
    return text

def pre_process_tweets(text):
  text=p.clean(text)  
  text = re.sub('[:@]', '',text)
  return text

In [ ]:
def detect_mental_health(user):
  max_tweets = 150
  tweets = tweepy.Cursor(api.user_timeline,id=user).items(max_tweets)
  tweets_list = [[tweet.text, tweet.created_at, tweet.id_str, tweet.user.screen_name, tweet.coordinates, tweet.place, tweet.retweet_count, tweet.favorite_count, tweet.lang, tweet.source, tweet.in_reply_to_status_id_str, tweet.in_reply_to_user_id_str, tweet.is_quote_status] for tweet in tweets]
  # Creation of dataframe from tweets_list
  tweets_df = pd.DataFrame(tweets_list,columns=['Tweet Text', 'Tweet Datetime', 'Tweet Id', 'Twitter @ Name', 'Tweet Coordinates', 'Place Info', 'Retweets', 'Favorites', 'Language', 'Source', 'Replied Tweet Id', 'Replied Tweet User Id Str', 'Quote Status Bool'])
  # Checks if there are coordinates attached to tweets, if so extracts them
  tweets_df['Tweet Coordinates'] = tweets_df.apply(extract_coordinates,axis=1) 
  # Checks if there is place information available, if so extracts them
  tweets_df['Place Info'] = tweets_df.apply(extract_place,axis=1)
  tweets_df['Tweet Text']=tweets_df['Tweet Text'].apply(pre_process_tweets)
  tweets_df['Tweet Datetime'] = pd.to_datetime(tweets_df['Tweet Datetime'])
  df=pd.DataFrame()
  df = (tweets_df.set_index('Tweet Datetime').between_time('21:00:00', '04:00:00').reset_index().reindex(columns=df.columns))
  night_tweets=(len(df)/150)*100
  tokenizer = Tokenizer(num_words=100000, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
  type_dict={}
  severity_dict={}
  for index,row in tweets_df.iterrows():
    tweet=[row['Tweet Text']]
    seq = tokenizer.texts_to_sequences(tweet)
    padded = pad_sequences(seq, maxlen=600)
    pred1 = cmodel.predict(padded)
    pred2=rmodel.predict(padded)
    alpha=cmodel_labels[np.argmax(pred1)]
    beta=rmodel_labels[np.argmax(pred2)]
    #alpha=random.choice(['anxiety and stress','relationship','domestic violence'])
    #beta=random.choice(rmodel_labels)
    if alpha in type_dict:
      type_dict[alpha]+=1
    else:
      type_dict[alpha]=1

    if beta in severity_dict:
      severity_dict[beta]+=1
    else:
      severity_dict[beta]=1
    a=max(type_dict.items(), key=operator.itemgetter(1))[0]
    b=max(severity_dict.items(), key=operator.itemgetter(1))[0]
    c=night_tweets
    print("Type of deprssion: ",a)
    print("Severity: ",b)
    print("Percentage of tweets at night time:",str(c)+"%")
    return a,b,c









In [ ]:
a,b,c=detect_mental_health('iam_kryystal')

Type of deprssion:  anxiety
Severity:  Concerned
Percentage of tweets at night time: 2.0%


In [ ]:
a,b,c

('anxiety', 'Concerned', 2.0)

In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))


https://o08xyhanqk-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [ ]:
from flask import Flask, render_template, request
from datetime import date 
from flask import session
import math
day=math.ceil(4.2)


app = Flask(__name__, template_folder='drive/My Drive/template',static_folder='drive/My Drive/static')
app.secret_key = "super secret key"
@app.route("/")
def home():  
  return render_template('login.html')

@app.route('/form_login',methods=['POST','GET'])
def login():
  name1=request.form['username']
  dep_type,severity,insomnia=detect_mental_health(name1)
  day=100-insomnia
  insomnia=math.ceil(insomnia)
  day=math.ceil(day)
  session['severity'] = severity
  #'anxiety and stress','relationship issues','domestic violence' 
  return render_template('new_index.html',name_input=name1,dep_type=dep_type,severity=severity,insomnia=insomnia,day=day)

@app.route("/contact")
def contact():  
  severity = session.get('severity')
  today = date.today() 
  d={'Safe to ignore':20,'Concerned':50,'Strongly Concerened':90}
  l=[[today,56]]
  return render_template('contact.html')


@app.route("/connect")
def connect():
  return render_template('connect.html')

@app.route("/work")
def work():
  return render_template('work.html')

  
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Feb/2021 12:08:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2021 12:08:54] "GET /static/bg1.png HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2021 12:08:54] "GET /static/avatar1.png HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2021 12:08:54] "GET /static/wave1.png HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2021 12:08:56] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [25/Feb/2021 12:13:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2021 12:13:43] "GET /static/wave1.png HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2021 12:13:43] "GET /static/bg1.png HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2021 12:13:43] "GET /static/avatar1.png HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2021 12:13:47] "GET /favicon.ico HTTP/1.1" 404 -


Type of deprssion:  anxiety
Severity:  Concerned
Percentage of tweets at night time: 2.0%


127.0.0.1 - - [25/Feb/2021 12:14:11] "POST /form_login HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2021 12:14:12] "POST /form_login HTTP/1.1" 200 -


Type of deprssion:  anxiety
Severity:  Concerned
Percentage of tweets at night time: 2.0%


127.0.0.1 - - [25/Feb/2021 12:14:13] "GET /static/owl-carousel.min.css HTTP/1.1" 404 -
127.0.0.1 - - [25/Feb/2021 12:14:13] "GET /static/aos.css HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2021 12:14:13] "GET /static/bootstrap.min.css HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2021 12:14:13] "GET /static/templatemo-digital-trend.css HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2021 12:14:14] "GET /static/font-awesome.min.css HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2021 12:14:14] "GET /static/owl.theme.default.min.css HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2021 12:14:15] "GET /fonts/Plain-Light.woff2 HTTP/1.1" 404 -
127.0.0.1 - - [25/Feb/2021 12:14:15] "GET /fonts/Plain-Regular.woff2 HTTP/1.1" 404 -
127.0.0.1 - - [25/Feb/2021 12:14:15] "GET /static/js/jquery.min.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2021 12:14:15] "GET /fonts/Plain-Light.woff HTTP/1.1" 404 -
127.0.0.1 - - [25/Feb/2021 12:14:15] "GET /static/js/smoothscroll.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Feb/2021 12:14:15] "GET /static/js/custom.js 

In [ ]:
#!pip install flask-ngrok

In [ ]:
import smtplib  
# creates SMTP session 
email = smtplib.SMTP('smtp.gmail.com', 587) 
  
# TLS for security 
email.starttls() 
  
# authentication
# compiler gives an error for wrong credential. 
email.login("yamuna.17cs@kct.ac.in", "Anjushana@123") 
  
# message to be sent 
message = "Critical situation"
  
# sending the mail 
email.sendmail("yamuna.17cs@kct.ac.in", "yamunaravi36@gmail.com", message) 
  
# terminating the session 
email.quit()

In [ ]:
d={'Safe to ignore':20,'Concerned':50,'Strongly Concerened':90}
severity='Concerned'
d[severity]